In [97]:
# Load problem from data
import numpy as np
import pandas as pd

from desdeo_problem.problem import IntegerVariable, VectorObjective, MOProblem

data = pd.read_fwf("~/Downloads/example_data.txt")
data = data.iloc[:, 1:]

# Assume that each unit has strategies between min and max without missing intermediate values
min_strategies = data.groupby("unit").min()["schedule"].values
max_strategies = data.groupby("unit").max()["schedule"].values

# Global min and max for each strategy
min_strategy_g = 1
max_strategy_g = np.max(max_strategies)

n_objectives = 7
n_variables = 100

In [211]:
# Define variables
var_names = [f"Unit {i+1}" for i in range(n_variables)]
vars = [IntegerVariable(var_names[i], initial_value=0, lower_bound=min_strategy_g-1, upper_bound=max_strategy_g-1) for i in range(n_variables)]

In [212]:
# Define a lookup table using the df to speed up evaluation
lookup_t = data.copy()
lookup_t = lookup_t.iloc[:, 2:]
units, schedules = data["unit"], data["schedule"]
lookup_t.insert(0, "unit & schedule", list(zip(units, schedules)))
len(lookup_t)

893

In [213]:
# df.loc[df['column_name'] != some_value]
# lookup_t.loc[lookup_t["unit & schedule"] == (1,60)].iloc[:, 1:].values
table = np.full((n_variables, max_strategy_g, n_objectives), np.inf)
table[units.values-1, schedules.values-1] = data.iloc[:, 2:]

In [226]:
# Define objective
def objective(x: np.ndarray, table: np.ndarray = table) -> np.ndarray:
    x = np.atleast_2d(x)
    if x.ndim == 2:
        # int values
        units = np.arange(x.shape[1], dtype=int)
        # minus because original problem is to be maximized
        return -table[units, x, :].sum(axis=1)
        
    elif x.ndim == 3:
        # vector of 1s and 0s
        print(x)
        pass
    print(x.shape)

# Objective names
objective_names = data.columns[2:]

objective = VectorObjective(objective_names, objective)
objective.evaluate(np.vstack((np.zeros((1, 100), dtype=int), np.ones((1, 100), dtype=int))))

ObjectiveEvaluationResults(objectives=(array([-221564.95,   -3846.59,   -9867.62,   -3168.64,  -10610.84,
         -2298.76,  -12585.24]), array([-inf, -inf, -inf, -inf, -inf, -inf, -inf])), uncertainity=array([[nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan]]))

In [209]:
#zeros = np.zeros((2,100), dtype=int)
#units = np.arange(100, dtype=int)
#table[units, zeros, :].sum(axis=1)

In [210]:
objective(np.ones(100, dtype=int))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


array([[inf, inf, inf, inf, inf, inf, inf]])

In [ ]:
# Define problem
problem = 